In [1]:
import pandas as pd
df = pd.read_csv("creditcardhist.csv")

In [5]:
df.head()
# df.info()
# df.isnull().sum()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [2]:
drop_cols = [
    "ID",
    "default.payment.next.month",
    "PAY_AMT1","PAY_AMT2","PAY_AMT3","PAY_AMT4","PAY_AMT5","PAY_AMT6",
    "BILL_AMT1","BILL_AMT2"
    
]
df_clean = df.drop(drop_cols,axis=1)
df_clean.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6
0,20000.0,2,2,1,24,2,2,-1,-1,-2,-2,689.0,0.0,0.0,0.0
1,120000.0,2,2,2,26,-1,2,0,0,0,2,2682.0,3272.0,3455.0,3261.0
2,90000.0,2,2,2,34,0,0,0,0,0,0,13559.0,14331.0,14948.0,15549.0
3,50000.0,2,2,1,37,0,0,0,0,0,0,49291.0,28314.0,28959.0,29547.0
4,50000.0,1,2,1,57,-1,0,-1,0,0,0,35835.0,20940.0,19146.0,19131.0


In [3]:
X = df_clean.drop("LIMIT_BAL",axis=1)
y = df_clean["LIMIT_BAL"] # splittiing data into X and y..

In [4]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(
    X,y,test_size=0.2,random_state=42
)

In [5]:
# standardization of value for accurate prediction so, that higher value do not affect the result..
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [6]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train_scaled,y_train)

,"fit_intercept fit_intercept: bool, default=TrueWhether to calculate the intercept for this model. If setto False, no intercept will be used in calculations(i.e. data is expected to be centered).",True
,"copy_X copy_X: bool, default=TrueIf True, X will be copied; else, it may be overwritten.",True
,"tol tol: float, default=1e-6The precision of the solution (`coef_`) is determined by `tol` whichspecifies a different convergence criterion for the `lsqr` solver.`tol` is set as `atol` and `btol` of :func:`scipy.sparse.linalg.lsqr` whenfitting on sparse training data. This parameter has no effect when fittingon dense data... versionadded:: 1.7",1e-06
,"n_jobs n_jobs: int, default=NoneThe number of jobs to use for the computation. This will only providespeedup in case of sufficiently large problems, that is if firstly`n_targets > 1` and secondly `X` is sparse or if `positive` is setto `True`. ``None`` means 1 unless in a:obj:`joblib.parallel_backend` context. ``-1`` means using allprocessors. See :term:`Glossary ` for more details.",None
,"positive positive: bool, default=FalseWhen set to ``True``, forces the coefficients to be positive. Thisoption is only supported for dense arrays.For a comparison between a linear regression model with positive constraintson the regression coefficients and a linear regression without such constraints,see :ref:`sphx_glr_auto_examples_linear_model_plot_nnls.py`... versionadded:: 0.24",False


In [23]:
y_pred = lr.predict(X_test_scaled)
y_pred

array([103054.69318782, 208204.62447457, 150298.14538368, ...,
       169870.6167716 , 164790.55898714, 293385.01991505])

In [16]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
import numpy as np

In [17]:
print("MAE  :", mean_absolute_error(y_test, y_pred))
print("RMSE :", np.sqrt(mean_squared_error(y_test, y_pred)))
print("R2   :", r2_score(y_test, y_pred))


MAE  : 83549.76608459618
RMSE : 107725.63723140495
R2   : 0.3111362711596556


In [22]:
coef_df = pd.DataFrame({
    "Features": X.columns,
    "Coefficient": lr.coef_
}).sort_values(by="Coefficient",ascending=False)
coef_df


,Features,Coefficient
13,BILL_AMT6,15369.240178
12,BILL_AMT5,15288.858830
3,AGE,14693.818168
10,BILL_AMT3,13082.709128
11,BILL_AMT4,11127.194492
0,SEX,1614.738994
8,PAY_5,-2607.286806
7,PAY_4,-7530.887795
2,MARRIAGE,-8443.915237
6,PAY_3,-9650.940886


In [26]:
new_customer = pd.DataFrame({
    "SEX": [2],
    "EDUCATION": [2],
    "MARRIAGE": [1],
    "AGE": [30],
    "PAY_0": [0],
    "PAY_2": [0],
    "PAY_3": [0],
    "PAY_4": [0],
    "PAY_5": [0],
    "PAY_6": [0],
    "BILL_AMT3": [20000],
    "BILL_AMT4": [22000],
    "BILL_AMT5": [21000],
    "BILL_AMT6": [23000]
})

new_customer_scaled = scaler.transform(new_customer)
predicted_limit = lr.predict(new_customer_scaled)
print("Predict Credit Limit : ",predicted_limit[0])


Predict Credit Limit :  137647.52834327397


In [7]:
# Save the model and scaler for the Flask app
import pickle

with open('model.pkl', 'wb') as f:
    pickle.dump(lr, f)

with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print("Model and scaler saved successfully!")
print(f"Model intercept: {lr.intercept_}")
print(f"Model coefficients: {lr.coef_}")

Model and scaler saved successfully!
Model intercept: 167226.65333333332
Model coefficients: [  1614.73899393 -25836.43937497  -8443.91523688  14693.8181676
 -13178.54046267 -15933.32449612  -9650.94088646  -7530.88779523
  -2607.28680563 -13297.07044596  13082.70912769  11127.19449152
  15288.85883031  15369.24017772]
